In [1]:
import sys

In [4]:
import pandas as pd
import os
from src.clustering_help_functions import create_clustering_object, define_type_of_editing, format_clustering_modification_dictionary, create_mapping_df
from src.reading_data import read_json_as_dict
from src.google_drive import get_gsheet_tab
from ml_data_bee.folder_file_loaders.dd.downloader import DdFileDownloader
from ml_data_bee.folder_file_loaders.goe.uploader import GoeFileUploader
from ml_clustering.clustering import ClusterEditor,ClusteringUtils,Clustering, AutoClustering


In [ ]:
# Configurations
clustering_modifications_file =


In [9]:
# Create dataset

# saved_file = "/Users/kleomeniskogias/DataspellProjects/GO-enterprise/clients/pepsi/PEPC0036/data/temp/bev_need_states_global_with_global_modifications_04102022.csv"
# need_states_global = pd.read_csv(saved_file, index_col=False).drop(columns = 'sub_cluster')
# print(need_states_global.shape)
# # Raw data
# client = 'pepsi'
# project = 'PEPC0036'
# release = '2022-06-24-16-51-55'  # Check that this release is the same wiht the one that I an using in the Pipeline
# file_name = 'beverage_logs_unstacked'
# print(client, project, file_name)
# cols_to_fetch = ['submission_id', 'mood_coded', 'where_are_you_having_your_food_drink', 'activities_while_consuming', 'created_at_grouped', 'level1_ontology_from_pepsi_bev_3_1', 'level3_ontology_from_pepsi_bev_3_1']
# bev_data = DdFileDownloader(client=client, project=project, local_base_path='../temp_data/', ).get_latest_snapshot(
#     file_name=file_name, release=release)[cols_to_fetch]
# print(bev_data.shape)
# need_states_global= need_states_global.merge(bev_data, on='submission_id')
# print(need_states_global.shape)

In [11]:
input_df = pd.read_csv("/Users/kleomeniskogias/DataspellProjects/GO-enterprise/clients/pepsi/PEPC0036/data/temp/Beverage substates with market level modifications_29092022.csv")
print(input_df.shape)

(126176, 27)


In [12]:
input_df.head()

,Unnamed: 0,level_0,index,submission_id,need_state,sub_cluster_init,macro_pillar,created_at,open_text,l1,...,needs list,mood_coded,where_are_you_having_your_food_drink,activities_while_consuming,created_at_grouped,level1_ontology_from_pepsi_bev_3_1,level3_ontology_from_pepsi_bev_3_1,SB_needstates,global_subcluster_number,global_subcluster_name
0,0,0,0,15310776,Hydrate,1.0,Hydrate,2021-01-01 01:13:23.879,"[""I was thirsty""]","[""i was really thirsty""]",...,['To quench my thirst'],"[""Neutral / fine""]",At my home,"[""Spending time with family"", ""Watching movies...",12am-3am,"[""Soft drinks""]","[""Dilutes / squash / cordial""]",Refresh,17,Thirst Quench
1,1,1,1,15310935,Fuel,0.0,Fuel,2021-01-01 01:14:20.726,"[""It\u2019s was on discount at the supermarket...","[""buy"", ""could"", ""so thought"", ""liked the can""...",...,['Good value for money'],"[""Great / amazing""]",At my home,"[""Talking / chatting"", ""Spending time with fam...",12am-3am,"[""Soft drinks""]","[""Diet carbonated soft drink (unspecified)""]",Routine,9,Easy Fuel
2,2,2,2,15310987,Hydrate,1.0,Hydrate,2021-01-01 01:25:36.821,"[""I\u2019m really thirsty""]","[""i was really thirsty""]",...,['To quench my thirst'],"[""Great / amazing""]",At my home,"[""Spending time with partner / on a date""]",12am-3am,"[""Soft drinks""]","[""Dilutes / squash / cordial""]",Refresh,17,Thirst Quench
3,3,3,3,15311135,Health,0.0,Health,2021-01-01 01:23:21.880,"[""It is sugar free and no calories and they se...","[""right drink"", ""water or sugar-free"", ""no cal...",...,"['No / low sugar', 'Low in (/zero) calories']","[""Content / upbeat""]",At my home,"[""Spending time with partner / on a date""]",12am-3am,"[""Soft drinks""]","[""Diet carbonated soft drink (unspecified)""]",Keeps health balanced,37,Sure & Easy
4,4,4,4,15311215,Fuel,1.0,Fuel,2021-01-01 01:28:21.114,"[""It gives energy and has high caffeine which ...","[""did something with caffeine"", ""bought energy""]",...,"['Gives me a boost / energy', 'Has caffeine']","[""Neutral / fine""]",At my home,"[""Relaxing / resting""]",12am-3am,"[""Soft drinks""]","[""Energy drink""]",Revive,7,Perk Up


In [7]:
submission_mapping.head()

,id,cluster_id
0,15310776.0,18.0
1,15310935.0,9.0
2,15310987.0,18.0
3,15311135.0,30.0
4,15311215.0,6.0


In [8]:
need_states_global

,index,submission_id,need_state,sub_cluster_init,final_need_state,created_at,open_text,l1,l2,ontology_codes,...,needs_from_ontology,needs,location_name,weights,mood_coded,where_are_you_having_your_food_drink,activities_while_consuming,created_at_grouped,level1_ontology_from_pepsi_bev_3_1,level3_ontology_from_pepsi_bev_3_1
0,0,15310776,Hydrate,1.0,Hydrate,2021-01-01 01:13:23.879,"[""I was thirsty""]","[""i was really thirsty""]","[""i was really thirsty""]","[""I had a craving""]",...,"[""I had a craving""]","[""To quench my thirst""]",UK,1.626915,"[""Neutral / fine""]",At my home,"[""Spending time with family"", ""Watching movies...",12am-3am,"[""Soft drinks""]","[""Dilutes / squash / cordial""]"
1,1,15310935,Fuel,0.0,Fuel,2021-01-01 01:14:20.726,"[""It\u2019s was on discount at the supermarket...","[""liked the can"", ""so thought"", ""could"", ""buy""...","[""NOISE""]","[""Cheap / affordable""]",...,"[""Good value for money""]","[""Good value for money""]",UK,0.410371,"[""Great / amazing""]",At my home,"[""Talking / chatting"", ""Spending time with fam...",12am-3am,"[""Soft drinks""]","[""Diet carbonated soft drink (unspecified)""]"
2,2,15310987,Hydrate,1.0,Hydrate,2021-01-01 01:25:36.821,"[""I\u2019m really thirsty""]","[""i was really thirsty""]","[""i was really thirsty""]","[""I had a craving""]",...,"[""I had a craving""]","[""To quench my thirst""]",UK,0.577433,"[""Great / amazing""]",At my home,"[""Spending time with partner / on a date""]",12am-3am,"[""Soft drinks""]","[""Dilutes / squash / cordial""]"
3,3,15311135,Health,0.0,Health,2021-01-01 01:23:21.880,"[""It is sugar free and no calories and they se...","[""no calories"", ""water or sugar-free"", ""right ...","[""no calories"", ""sugar-free""]","[""It's a healthy choice"", ""To relax / relieve ...",...,"[""Is nutritious / healthy"", ""To relax & unwind""]","[""Low in (/zero) calories"", ""No / low sugar""]",UK,0.454904,"[""Content / upbeat""]",At my home,"[""Spending time with partner / on a date""]",12am-3am,"[""Soft drinks""]","[""Diet carbonated soft drink (unspecified)""]"
4,4,15311215,Fuel,1.0,Fuel,2021-01-01 01:28:21.114,"[""It gives energy and has high caffeine which ...","[""did something with caffeine"", ""bought energy""]","[""to give me energy"", ""caffeine related""]","[""To boost energy""]",...,"[""Gives me a boost / energy""]","[""Has caffeine"", ""Gives me a boost / energy""]",UK,0.343354,"[""Neutral / fine""]",At my home,"[""Relaxing / resting""]",12am-3am,"[""Soft drinks""]","[""Energy drink""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126171,19819,44581111,Indulge,1.0,Indulge,2022-05-28 12:57:10 +0000,"[""I like this drink""]","[""really like this drink""]","[""really like this drink""]","[""I like the taste""]",...,"[""Is tasty""]","[""To indulge""]",FR,1.559807,"[""Content / upbeat""]",Other,NaN,12pm-3pm,"[""Beer""]","[""Pale Lager / Pilsner""]"
126172,19820,44582257,Health,6.0,Health,2022-05-28 18:17:32 +0000,"[""To go with the meal""]","[""going with the meal""]","[""NOISE""]","[""To go with other food / drink""]",...,"[""Completes my meal""]","[""Completes my meal""]",FR,0.422429,"[""Content / upbeat""]",In a cafe or restaurant,NaN,6pm-9pm,"[""Cider / Perry""]",NaN
126173,19821,44612107,Indulge,4.0,Indulge,2022-05-29 18:49:03 +0000,"[""By desire by taste I like the ros\u00e9 grap...","[""not very strong"", ""desire for breakfast"", ""i...","[""has taste"", ""it is fresh""]","[""I like the taste""]",...,"[""Is tasty""]","[""Is Fresh / Fresh / freshly made"", ""Is tasty""]",FR,1.927009,"[""Content / upbeat""]",At my home,"[""Watching movies / TV / sports""]",6pm-9pm,"[""Wine""]",NaN
126174,19822,44748969,Hydrate,3.0,Hydrate,2022-05-30 17:30:11 +0000,"[""because I'm thirsty""]","[""i was really thirsty""]","[""i was really thirsty""]","[""To feel refreshed / hydrated""]",...,"[""To rehydrate my body""]","[""To quench my thirst""]",FR,0.665487,"[""Hopeful / optimistic""]",In a bar / pub,NaN,6pm-9p

In [ ]:
for modification in clustering_modifications_file:
    modification, cluster_names_dict= define_type_of_editing(modification, cluster_names_dict,clustering) # Type of editing
    d = format_clustering_modification_dictionary(modification)   # Modify the format of the dictionary
    # Apply modification
    if modification['from_cluster'] is not None:
        print(editor.select(selections=d,from_cluster=modification['from_cluster'],to_cluster=modification['to_cluster']))
        editor.reassign()